If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [3]:
!pip install datasets transformers[sentencepiece] sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [3]:
import transformers

print(transformers.__version__)

4.46.3


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [4]:
from transformers.utils import send_example_telemetry

send_example_telemetry("translation_notebook", framework="pytorch")

# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a translation task. We will use the [WMT dataset](http://www.statmt.org/wmt16/), a machine translation dataset composed from a collection of various sources, including news commentaries and parliament proceedings.

![Widget inference on a translation task](https://github.com/huggingface/notebooks/blob/master/examples/images/translation.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [80]:
model_checkpoint = "Helsinki-NLP/opus-mt-hi-en"

In [114]:
# from transformers import AutoTokenizer
# text = "रस्सी जल गयी, बल नहींगया"
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
# inputs = tokenizer(text, return_tensors="pt").input_ids
# from transformers import AutoModelForSeq2SeqLM

# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
# outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
# tokenizer.decode(outputs[0], skip_special_tokens=True)

'The rope burned, did not get force'

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) checkpoint.

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [81]:
# !pip install datasets --upgrade

In [82]:
!pip install evaluate

In [115]:
from datasets import load_dataset#, load_metric
from evaluate import load
raw_datasets = load_dataset("Helsinki-NLP/open_subtitles", lang1="en", lang2="hi")
metric = load("sacrebleu")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [116]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'meta', 'translation'],
        num_rows: 93016
    })
})

To access an actual element, you need to select a split first, then give an index:

In [117]:
raw_datasets["train"][0]

{'id': '0',
 'meta': {'year': 1948,
  'imdbId': 40522,
  'subtitleId': {'en': 4180294, 'hi': 4239106},
  'sentenceIds': {'en': [1], 'hi': [1]}},
 'translation': {'en': 'THE BICYCLE THIEF', 'hi': 'साइकिल चोर'}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [118]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [119]:
show_random_elements(raw_datasets["train"])

,id,meta,translation
0,83810,"{'year': 2017, 'imdbId': 2674358, 'subtitleId': {'en': 6974954, 'hi': 7056459}, 'sentenceIds': {'en': [38], 'hi': [41]}}","{'en': 'Thanks, Mimi.', 'hi': 'धन्यवाद, मिमी।'}"
1,14592,"{'year': 2004, 'imdbId': 363163, 'subtitleId': {'en': 3468399, 'hi': 4398107}, 'sentenceIds': {'en': [861], 'hi': [968]}}","{'en': 'Be ruthless.', 'hi': 'क्रूर बनो.'}"
2,3278,"{'year': 1983, 'imdbId': 84390, 'subtitleId': {'en': 4244114, 'hi': 4228820}, 'sentenceIds': {'en': [451], 'hi': [502]}}","{'en': 'We must act quickly to be able to sleep peacefully.', 'hi': 'हमे जल्दी कुछ करना चाहिए ताकी हम चैन से सो सकें.'}"
3,36048,"{'year': 2012, 'imdbId': 2182241, 'subtitleId': {'en': 5046814, 'hi': 5046815}, 'sentenceIds': {'en': [45], 'hi': [49]}}","{'en': 'I want to serve Jesus.', 'hi': 'मैं यीशु की सेवा करना चाहते हैं.'}"
4,32098,"{'year': 2011, 'imdbId': 1941569, 'subtitleId': {'en': 5456720, 'hi': 5456723}, 'sentenceIds': {'en': [639], 'hi': [875]}}","{'en': 'And all my friends came to my rescue and helped me.', 'hi': 'मेरे सभी दोस्त मेरे बचाव के लिए आ गए और उन्होनें मेरी सहायता की.'}"


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [120]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [121]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [122]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [123]:
!pip install sacremoses

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [124]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [125]:
tokenizer("Hello, this one sentence!")

{'input_ids': [9865, 2, 95, 121, 50, 15817, 8041, 4559, 58, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [126]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[9865, 2, 95, 121, 50, 15817, 8041, 4559, 58, 0], [273, 25, 138, 5431, 50, 15817, 8041, 4559, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [127]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[9865, 2, 95, 121, 8433, 58, 0], [273, 25, 411, 8433, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [128]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [129]:
prefix

''

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [130]:
max_input_length = 128
max_target_length = 128
source_lang = "hi"
target_lang = "en"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [131]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[13860, 7208, 0], [8133, 24085, 123, 22, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1, 1, 1]], 'labels': [[2444, 1051, 14628, 2145, 713, 8936, 23595, 382, 610, 1332, 0], [1962, 1800, 5755, 22, 0]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [132]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/93016 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [133]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [134]:
batch_size = 64
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    #evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    #per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    logging_steps=500,
    save_total_limit=3,
    num_train_epochs=5,
    #predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/marian-finetuned-en-to-ro"` or `"huggingface/marian-finetuned-en-to-ro"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [135]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [136]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [137]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    #eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-137-cf7f6bb45632>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


We can now finetune our model by just calling the `train` method:

In [138]:
trainer.train()

Step,Training Loss
500,1.078000
1000,1.078600
1500,1.072100
2000,0.963000
2500,0.961000
3000,0.940600
3500,0.882700
4000,0.881100
4500,0.869200
5000,0.827700


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=7270, training_loss=0.9103648714383006, metrics={'train_runtime': 1434.8109, 'train_samples_per_second': 324.14, 'train_steps_per_second': 5.067, 'total_flos': 3656381454876672.0, 'train_loss': 0.9103648714383006, 'epoch': 5.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [141]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Ellight/opus-mt-hi-en-finetuned-hi-to-en/commit/2cc27d95a3bd7ee8bab1aa2e36d9c0500017d7dc', commit_message='End of training', commit_description='', oid='2cc27d95a3bd7ee8bab1aa2e36d9c0500017d7dc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ellight/opus-mt-hi-en-finetuned-hi-to-en', endpoint='https://huggingface.co', repo_type='model', repo_id='Ellight/opus-mt-hi-en-finetuned-hi-to-en'), pr_revision=None, pr_num=None)

In [142]:
from transformers import AutoTokenizer
text = "रस्सी जल गयी, बल नहींगया"
tokenizer = AutoTokenizer.from_pretrained("Ellight/opus-mt-hi-en-finetuned-hi-to-en")
inputs = tokenizer(text, return_tensors="pt").input_ids
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("Ellight/opus-mt-hi-en-finetuned-hi-to-en")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

"The rope burned, they weren't strong."

In [143]:
from transformers import AutoTokenizer
text = "एक म्यान में दो तलवारें नहीं समातीं"
tokenizer = AutoTokenizer.from_pretrained("Ellight/opus-mt-hi-en-finetuned-hi-to-en")
inputs = tokenizer(text, return_tensors="pt").input_ids
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("Ellight/opus-mt-hi-en-finetuned-hi-to-en")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Do not cover two swords in an operation'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```

In [1]:
# from huggingface_hub import notebook_login

# notebook_login()

In [19]:
# from datasets import load_dataset

# ds = load_dataset("cfilt/iitb-english-hindi")

In [20]:
# ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [6]:
# from transformers import AutoTokenizer

# checkpoint = "google-t5/t5-small"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [8]:
# tokenizer.tokenize("रस्सी जल गयी, बल नहींगया")
# tokenizer.tokenize("Virat Kohli is a great batsmen")

['▁Vi', 'rat', '▁Kohl', 'i', '▁is', '▁', 'a', '▁great', '▁bat', 's', 'men']

In [24]:
# source_lang = "hi"
# target_lang = "en"
# prefix = "translate Hindi to English: "


# def preprocess_function(examples):
#     inputs = [prefix + example[source_lang] for example in examples["translation"]]
#     targets = [example[target_lang] for example in examples["translation"]]
#     model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
#     return model_inputs

In [26]:
# tokenized_books = ds_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [27]:
# from transformers import DataCollatorForSeq2Seq
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [28]:
# !pip install evaluate

In [29]:
# import evaluate

# metric = evaluate.load("sacrebleu")

In [30]:
# import numpy as np


# def postprocess_text(preds, labels):
#     preds = [pred.strip() for pred in preds]
#     labels = [[label.strip()] for label in labels]

#     return preds, labels


# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {"bleu": result["score"]}

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     result = {k: round(v, 4) for k, v in result.items()}
#     return result

In [35]:
# from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [36]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="t5_small_iitb_model",
#     #eval_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     #per_device_eval_batch_size=16,
#     weight_decay=0.01,
#     logging_steps=100,
#     save_total_limit=3,
#     num_train_epochs=2,
#     predict_with_generate=True,
#     fp16=True, #change to bf16=True for XPU
#     push_to_hub=True,
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_books,#["train"],
#     #eval_dataset=tokenized_books["test"],
#     processing_class=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

Step,Training Loss
100,4.893200
200,4.289000
300,4.077100
400,4.003100
500,3.886700
600,3.848900
700,3.842900
800,3.829700
900,3.877200
1000,3.732100


TrainOutput(global_step=12500, training_loss=3.4523283618164062, metrics={'train_runtime': 1432.3865, 'train_samples_per_second': 139.627, 'train_steps_per_second': 8.727, 'total_flos': 2441354626990080.0, 'train_loss': 3.4523283618164062, 'epoch': 2.0})

In [37]:
# trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Ellight/t5_small_iitb_model/commit/c6f5a131fb3aab568e37c8c23cab9c1c5c261fd9', commit_message='End of training', commit_description='', oid='c6f5a131fb3aab568e37c8c23cab9c1c5c261fd9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ellight/t5_small_iitb_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Ellight/t5_small_iitb_model'), pr_revision=None, pr_num=None)

In [41]:
# text = "translate Hindi to English: रस्सी जल गयी, बल नहींगया"

In [39]:
# from transformers import pipeline

# translator = pipeline("translation_xx_to_yy", model="Ellight/t5_small_iitb_model")
# translator(text)

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'translation_text': 'The corresponding file, the corresponding file'}]

In [42]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("Ellight/t5_small_iitb_model")
# inputs = tokenizer(text, return_tensors="pt").input_ids
# from transformers import AutoModelForSeq2SeqLM

# model = AutoModelForSeq2SeqLM.from_pretrained("Ellight/t5_small_iitb_model")
# outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
# tokenizer.decode(outputs[0], skip_special_tokens=True)

"Revert the 'daily' window from the 'daily' window."